## Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy

from point_cloud_segmentation.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from point_cloud_segmentation.visualizing.visualise import *
from point_cloud_segmentation.io.pcd_file import *


2026-02-05 11:58:58.655 | INFO     | point_cloud_segmentation.config:<module>:11 - PROJ_ROOT path is: D:\wkd\work\point-cloud-segmentation


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Loading data

In [2]:
# las = laspy.read(RAW_DATA_DIR / "wkd-house_1_8xds.laz")
# las = laspy.read(RAW_DATA_DIR / "wkd-house_2_8xds.laz")
# las = laspy.read(RAW_DATA_DIR / "wkd-half_house_2_8xds.laz")

pcd = las2pcd(RAW_DATA_DIR / "wkd-house_1_8xds.laz")

2026-02-05 12:00:47.426 | SUCCESS  | point_cloud_segmentation.io.pcd_file:las2pcd:19 - PointCloud loaded from: D:\wkd\work\point-cloud-segmentation\data\raw\wkd-house_1_8xds.laz
2026-02-05 12:00:47.430 | INFO     | point_cloud_segmentation.io.pcd_file:las2pcd:20 - PointCloud with 70775329 points.


## Downsampling

In [3]:
voxel_size = 0.05
ds_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(ds_pcd)

PointCloud with 524347 points.


In [4]:
# visualize_geometries(ds_pcd)

## Segmentation

In [5]:
segmented_objects = []
unsegmented_objects = []
object_n = 20

threshold = voxel_size # how far each point to be included
# threshold = voxel_size # how far each point to be included
iterations = 1000
# og_object = copy.deepcopy(pcd)
ds_pcd_np = np.array(ds_pcd.points)
processed_pcd = copy.deepcopy(ds_pcd)
print(ds_pcd_np)
print(ds_pcd_np.shape)
for i in range(object_n):
    # logger.info("Start plane RANSAC")
    eq, idx_inliers = processed_pcd.segment_plane(distance_threshold=threshold, ransac_n=3 ,num_iterations=iterations)
    processed_pcd_np = np.array(processed_pcd.points)

    # logger.info("Start getting inliers and outliers")
    inliers = processed_pcd_np[idx_inliers]

    pcd_np = np.array(processed_pcd.points)

    mask = np.ones(len(pcd_np), dtype=bool)
    mask[idx_inliers] = False
    outliers = pcd_np[mask]

    # logger.info("Start converting np to o3d pcd")
    outliers_pcd = o3d.geometry.PointCloud()
    outliers_pcd.points = o3d.utility.Vector3dVector(outliers)
    processed_pcd = copy.deepcopy(outliers_pcd)
    # visualize_geometries(outliers_pcd)

    inlier_pcd = o3d.geometry.PointCloud()
    inlier_pcd.points = o3d.utility.Vector3dVector(inliers)
    segmented_objects.append(inlier_pcd)
    print(inliers.shape)
    print(outliers.shape)
    if i >= object_n - 1:
        print("ASDASD")
        unsegmented_objects.append(outliers_pcd)

print(segmented_objects)
print(unsegmented_objects)
visualize_geometries(segmented_objects)
visualize_geometries(unsegmented_objects)
# visualize_geometries(segmented_objects, unsegmented_objects)


[[ 8.43475     7.008375    4.25775   ]
 [11.64196765  9.17401294 -1.9348586 ]
 [ 6.57854078 10.80543204 -1.2858835 ]
 ...
 [-0.448      10.8905      1.0705    ]
 [-0.3185     10.6125      1.1255    ]
 [-0.4195     11.074      -0.763     ]]
(524347, 3)
(42629, 3)
(481718, 3)
(39786, 3)
(441932, 3)
(21554, 3)
(420378, 3)
(22550, 3)
(397828, 3)
(11884, 3)
(385944, 3)
(13243, 3)
(372701, 3)
(10189, 3)
(362512, 3)
(8960, 3)
(353552, 3)
(16990, 3)
(336562, 3)
(19449, 3)
(317113, 3)
(10173, 3)
(306940, 3)
(13168, 3)
(293772, 3)
(7240, 3)
(286532, 3)
(7118, 3)
(279414, 3)
(6988, 3)
(272426, 3)
(6021, 3)
(266405, 3)
(8803, 3)
(257602, 3)
(7090, 3)
(250512, 3)
(6538, 3)
(243974, 3)
(5515, 3)
(238459, 3)
ASDASD
[PointCloud with 42629 points., PointCloud with 39786 points., PointCloud with 21554 points., PointCloud with 22550 points., PointCloud with 11884 points., PointCloud with 13243 points., PointCloud with 10189 points., PointCloud with 8960 points., PointCloud with 16990 points., PointCloud 

## Exporting

In [6]:
pcd2las(ds_pcd, PROCESSED_DATA_DIR / "laz_output.laz")


2026-02-05 12:01:05.616 | INFO     | point_cloud_segmentation.io.pcd_file:pcd2las:38 - Point cloud doesn`t colors
2026-02-05 12:01:05.780 | SUCCESS  | point_cloud_segmentation.io.pcd_file:pcd2las:56 - Successfully exported point cloud to D:\wkd\work\point-cloud-segmentation\data\processed\laz_output.laz
